In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
import os
import torch
import numpy as np
import torch.nn as nn
import scipy.io

In [3]:
import pickle
from dotenv import load_dotenv

load_dotenv()

PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), "../../.."))

PRINT_EVERY = 1000

## Start: Importing local packages. As I don't know how to run it as module
## with torchrun  (e.g., python -m trainer.Coronary_ddp_trainer)

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

from src.utils.utils import lp_error
from src.data import IBM_data_loader
from src.data.IBM_data_loader import IBM_data_loader

device = "cpu"  # torch.device("cuda" if torch.cuda.is_available() else "cpu")

# file_path = "/home/vlq26735/saiful/afrah/datasets/IBM/IB_PINN3.mat"

# h=  0.498715
# k=  0.6851750000000001
# rx=  0.209685
# ry=  0.21008499999999997

In [4]:
DATASET_PATH = "../../../data/Fluid_trainingData.mat"

Fluid_data = scipy.io.loadmat(DATASET_PATH)

# Load data
fluid = Fluid_data["Fluid_training"]
interface = Fluid_data["Solid_interface"]
solid = Fluid_data["Solid_points"]

In [5]:
train_dataloader = IBM_data_loader(DATASET_PATH, "cuda:0")


tstep = 101
xstep = 102
ystep = 102

FluidData: self.txy_fluid.shape=torch.Size([1050804, 3]),  self.mean_x=tensor([ 5.0000e-01,  5.0000e-01,  4.9998e-01,  4.4984e-03,  1.5413e-04,
         3.8639e-03, -8.2371e-04, -4.6937e-04], device='cuda:0'), self.std_x=tensor([0.2915, 0.2901, 0.2901, 0.1987, 0.1269, 0.1122, 1.2655, 1.2822],
       device='cuda:0')
SolidData: self.txy_solid_points.shape=torch.Size([143780, 3]),  self.mean_x=tensor([ 0.5044,  0.4678,  0.6521, -0.0124,  0.0204, -0.0232,  0.0282,  0.0481],
       device='cuda:0'), self.std_x=tensor([0.2904, 0.1506, 0.1444, 0.1219, 0.1065, 0.0854, 2.8319, 3.0344],
       device='cuda:0')
FluidSolidInterfaceData: self.txy_interface.shape=torch.Size([25452, 3]),  self.mean_x=tensor([ 5.0001e-01,  4.6873e-01,  6.5305e-01, -1.0230e-02,  1.8494e-02,
        -6.9603e-03, -7.7546e+01,  5.0739e+01], device='cuda:0'), self.std_x=tensor([2.9155e-01, 1.7971e-01, 1.7300e-01, 1.4451e-01, 1.3330e-01, 1.8181e-01,
        9.7830e+02, 7.2563e+02], device='cuda:0')


In [7]:
# Initialize fluid velocity (u, v) and pressure (p)
txy_fluid = train_dataloader.fluid_data.txy_fluid_points[:, 0:3].cpu().numpy()

u_fluid = train_dataloader.fluid_data.uvp_fluid_points[:, 0].cpu().numpy()
v_fluid = train_dataloader.fluid_data.uvp_fluid_points[:, 1].cpu().numpy()
p_fluid = train_dataloader.fluid_data.uvp_fluid_points[:, 2].cpu().numpy()
fx_fluid = train_dataloader.fluid_data.uvp_fluid_points[:, 3].cpu().numpy()
fy_fluid = train_dataloader.fluid_data.uvp_fluid_points[:, 4].cpu().numpy()

txy_interface = train_dataloader.interface_data.txy_interface[:, 0:3].cpu().numpy()

u_interface = train_dataloader.interface_data.uvp_interface[:, 0].cpu().numpy()
v_interface = train_dataloader.interface_data.uvp_interface[:, 1].cpu().numpy()
p_interface = train_dataloader.interface_data.uvp_interface[:, 2].cpu().numpy()
fx_interface = train_dataloader.interface_data.uvp_interface[:, 3].cpu().numpy()
fy_interface = train_dataloader.interface_data.uvp_interface[:, 4].cpu().numpy()

# # Select indices where t_fluid = t_interface, x_fluid = x_interface, and y_fluid = y_interface
# indices = np.where(
#     (t_fluid[:, None] == t_interface)
#     & (x_fluid[:, None] == x_interface)
#     & (y_fluid[:, None] == y_interface)
# )

In [8]:
def delta_function_2D(x_eulerian, X_lagrangian, epsilon=0.1):

    # Compute differences in x and y directions
    diff_x = x_eulerian[0] - X_lagrangian[0]
    diff_y = x_eulerian[1] - X_lagrangian[1]

    # Apply Gaussian kernel separately in x and y directions
    delta_x = (np.exp(-(diff_x**2 + diff_y**2) / (2 * epsilon**2))) / (
        epsilon**2 * (2.0 * np.pi)
    )

    # Multiply the 1D Gaussians to form the 2D delta function
    delta_value = delta_x  # * delta_y

    return delta_value


def spread_lagrangian_force_to_eulerian(
    lagranian_point, X_eulerian, Fx_lagrangian, Fy_lagrangian, epsilon=0.1
):

    f_x_eulerian = 0.0
    f_y_eulerian = 0.0
    for index in range(len(X_eulerian)):
        eulerian_point = X_eulerian[index]
        F_x_lag = Fx_lagrangian[index]
        F_y_lag = Fy_lagrangian[index]

        delta_value = delta_function_2D(eulerian_point, lagranian_point, epsilon)

        # Spread the Lagrangian force to the Eulerian grid point
        f_x_eulerian += delta_value * F_x_lag
        f_y_eulerian += delta_value * F_y_lag

    return f_x_eulerian, f_y_eulerian

In [22]:
# Bilinear interpolation to spread Lagrangian forces to Eulerian grid
def bilinear_interpolate_force_to_eulerian(
    X_lagrangian,
    F_x_lagrangian,
    F_y_lagrangian,
    size,
    dx,
    dy,
):
    """
    Interpolates Lagrangian forces to the Eulerian grid using bilinear interpolation.

    Args:
        X_lagrangian, Y_lagrangian (numpy.ndarray): Lagrangian point positions (x, y).
        F_x_lagrangian, F_y_lagrangian (numpy.ndarray): Lagrangian force components (x, y).
        f_x_eulerian, f_y_eulerian (numpy.ndarray): Eulerian force field to be updated (x, y).
        dx, dy (float): Eulerian grid spacing.

    Returns:
        Updated Eulerian force field f_x_eulerian, f_y_eulerian.
    """
    nx, ny = size * 2, size * 2
    f_x_eulerian = np.zeros((size, size))  # Eulerian x-forces
    f_y_eulerian = np.zeros((size, size))  # Eulerian y-forces

    for index in range(len(X_lagrangian)):

        # for lag_idx, (x_lag, y_lag, F_x_lag, F_y_lag) in enumerate(
        #     zip(X_lagrangian, Y_lagrangian, F_x_lagrangian, F_y_lagrangian)
        # ):
        # Find the lower-left Eulerian grid point surrounding the Lagrangian point
        x_lag = X_lagrangian[index, 0]
        y_lag = X_lagrangian[index, 1]
        F_x_lag = F_x_lagrangian[index]
        F_y_lag = F_y_lagrangian[index]

        i = int(x_lag // dx)
        j = int(y_lag // dy)

        # Ensure the Lagrangian point is within the Eulerian grid bounds
        if i < 0 or i >= nx - 1 or j < 0 or j >= ny - 1:
            continue

        # Calculate fractional distances in the x and y directions
        alpha_x = (x_lag - i * dx) / dx
        alpha_y = (y_lag - j * dy) / dy

        # Compute the interpolation weights for the surrounding four Eulerian grid points
        w_ij = (1 - alpha_x) * (1 - alpha_y)
        w_i1j = alpha_x * (1 - alpha_y)
        w_ij1 = (1 - alpha_x) * alpha_y
        w_i1j1 = alpha_x * alpha_y

        # Spread the Lagrangian force to the four nearest Eulerian grid points
        f_x_eulerian[i, j] += w_ij * F_x_lag
        f_x_eulerian[i + 1, j] += w_i1j * F_x_lag
        f_x_eulerian[i, j + 1] += w_ij1 * F_x_lag
        f_x_eulerian[i + 1, j + 1] += w_i1j1 * F_x_lag

        f_y_eulerian[i, j] += w_ij * F_y_lag
        f_y_eulerian[i + 1, j] += w_i1j * F_y_lag
        f_y_eulerian[i, j + 1] += w_ij1 * F_y_lag
        f_y_eulerian[i + 1, j + 1] += w_i1j1 * F_y_lag

    return f_x_eulerian, f_y_eulerian

In [34]:
# Initialize the Eulerian grid and Lagrangian points
nx, ny = 102, 102  # Grid resolution
dx, dy = 1.0 / nx, 1.0 / ny  # Grid spacing

time_step = 1
# Initialize velocity and force fields
u_eulerian = (
    u_fluid.reshape(tstep, xstep, ystep).transpose((0, 2, 1))[time_step, :, :].flatten()
)  # Eulerian u velocity
v_eulerian = (
    v_fluid.reshape(tstep, xstep, ystep).transpose((0, 2, 1))[time_step, :, :].flatten()
)  # Eulerian v velocity
Fx_lagrangian = (
    fx_fluid.reshape(tstep, xstep, ystep)
    .transpose((0, 2, 1))[time_step, :, :]
    .flatten()
)  # Eulerian x-force
Fy_lagrangian = (
    fy_fluid.reshape(tstep, xstep, ystep)
    .transpose((0, 2, 1))[time_step, :, :]
    .flatten()
)  # Eulerian y-force

time_fluid = txy_fluid[:, 0].reshape(tstep, -1)[time_step, :]
x_fluid = txy_fluid[:, 1].reshape(tstep, -1)[time_step, :]
y_fluid = txy_fluid[:, 2].reshape(tstep, -1)[time_step, :]

time_interface = txy_interface[:, 0].reshape(tstep, -1)[time_step, :]
x_interface = txy_interface[:, 1].reshape(tstep, -1)[time_step, :]
y_interface = txy_interface[:, 2].reshape(tstep, -1)[time_step, :]

X_eulerian = np.concatenate(
    [x_fluid.reshape(-1, 1), y_fluid.reshape(-1, 1)], axis=1
)  # Eulerian grid points

X_lagrangian = np.concatenate(
    [x_interface.reshape(-1, 1), y_interface.reshape(-1, 1)], axis=1
)  # Lagrangian points

real_fx_eulerian = fx_interface.reshape(tstep, -1)[time_step, :]  # Lagrangian x-forces
real_fy_eulerian = fy_interface.reshape(tstep, -1)[time_step, :]  # Lagrangian y-forces

fx_eulerian = real_fx_eulerian.shape[0]  # Force in x direction
fy_eulerian = real_fx_eulerian.shape[0]  # Force in y direction

size = real_fx_eulerian.shape[0] // 2
f_x_eulerian = np.zeros((size, size))  # Eulerian x-forces
f_y_eulerian = np.zeros((size, size))  # Eulerian y-forces

# # # Spread Lagrangian forces to Eulerian grid
# for index in range((X_lagrangian.shape[0])):
#     lagranian_point = X_lagrangian[index, :]
f_x_eulerian, f_y_eulerian = bilinear_interpolate_force_to_eulerian(
    X_lagrangian, Fx_lagrangian, Fy_lagrangian, size, dx, dy
)

# f_x_eulerian[index], f_y_eulerian[index] = spread_lagrangian_force_to_eulerian(
#     lagranian_point,
#     X_eulerian,
#     Fx_lagrangian,
#     Fy_lagrangian,
#     epsilon=0.1,
# )

In [32]:
f_x_eulerian = f_x_eulerian.reshape(-1, 1)
f_y_eulerian = f_y_eulerian.reshape(-1, 1)

In [35]:
f_x_eulerian.shape

(126, 126)

In [36]:
real_fx_eulerian.shape

(252,)

In [10]:
text = "RelL2_Error: "

u_error2 = lp_error(f_x_eulerian, real_fx_eulerian, (text + "U%"))
u_error2 = lp_error(f_y_eulerian, real_fy_eulerian, (text + "U%"))

RelL2_Error: U%  : 94.52 
RelL2_Error: U%  : 121.28 


In [11]:
import pandas as pd

# Reshape the array to match the original data structure
df_fx_eulerian = pd.DataFrame(real_fx_eulerian[:100].reshape(25, -1))

# Display the DataFrame
print(df_fx_eulerian)

            0           1          2           3
0   -4.790200  -11.393000  13.102000    7.218500
1  -17.184999   29.327000 -23.613001  -19.788000
2   -2.905400   52.549999  -4.653000  -20.385000
3   27.208000  -18.961000   3.171100  -17.443001
4  -30.275999   -0.122150  73.988998  -18.676001
5   10.933000  -25.128000  47.389000  -32.789001
6  -36.196999  -17.247000  -9.182800   66.667000
7  -12.021000    2.715400 -23.495001  -19.757000
8   27.018000  -21.028999 -21.028999  -25.756001
9  -34.257000   -8.140100 -33.782001  -10.540000
10  -2.848300  -23.858999  -1.522600  -14.419000
11  32.097000   47.992001 -26.934999  123.440002
12  -3.889200  -33.617001  64.679001  -10.550000
13 -21.134001   -9.182800 -12.964000   97.334999
14  50.644001   -1.132300 -61.888000  -21.580999
15 -20.634001  -44.316002  -4.653000   38.294998
16  -2.720400   -3.280200 -23.613001    2.762600
17  13.102000   -8.325700  92.747002  -10.995000
18  38.073002   -1.067100 -18.292000  -18.587999
19 -24.049999  113.5

In [12]:
import pandas as pd

# Reshape the array to match the original data structure
df_fx_eulerian = pd.DataFrame(f_x_eulerian[:100].reshape(25, -1))

# Display the DataFrame
print(df_fx_eulerian)

            0          1          2          3
0  -27.373263 -28.234636  29.196352  25.617751
1  -51.389255  40.977386 -15.137369 -72.965565
2   -8.506745  72.258659 -15.433190 -76.852122
3   34.819466  27.960542  -0.251703 -32.807812
4  -64.338728  -2.303086  62.870667 -78.025699
5   79.636558 -78.402564  37.120023 -61.463793
6    0.167848 -52.038314  19.660157  87.562147
7  -10.631380  23.685832 -78.797586 -47.196255
8   42.367626 -55.489079 -55.489079 -59.744399
9  -57.699329  67.371618 -74.901708  50.704146
10   2.856821 -50.329221   4.936641 -41.133856
11  81.444878  54.717027 -49.256696  87.061196
12  10.068403 -54.293974  80.508785 -38.550213
13 -58.946564  19.660157  39.451893  79.250851
14  75.175866   3.893281 -80.012693 -53.720931
15 -61.738491  -2.758238 -15.433190  53.097991
16  17.366378  -9.584021 -15.137369 -11.818547
17  29.196352  18.156757  76.580134 -38.746960
18  88.164786   7.012797 -67.347209 -45.800399
19 -49.128031  57.981569  12.034228  27.679907
20  38.355472

In [ ]:

def spread_lagrangian_force_to_eulerian(
    x_eulerian_pt,
    X_lagrangian,
    F_x_lagrangian,
    F_y_lagrangian,
    epsilon=0.1,
):


In [ ]:
# Interpolate Eulerian velocity to Lagrangian points
u_lagrangian, v_lagrangian = interpolate_eulerian_velocity_to_lagrangian(
    u_eulerian, v_eulerian, X_lagrangian, dx, dy
)

# Print results
print("Lagrangian velocities (u, v):")
print(u_lagrangian, v_lagrangian)

In [ ]:
FluidData: self.txy_fluid.shape=torch.Size([1014684, 3]),  self.mean_x=tensor([ 0.4998,  0.5012,  0.4945,  0.0049, -0.0007,  0.0049,  0.0046,  0.0071],
       device='cuda:0'), self.std_x=tensor([0.2917, 0.2930, 0.2917, 0.1992, 0.1266, 0.1091, 1.0603, 1.1419],
       device='cuda:0')
SolidData: self.txy_solid_points.shape=torch.Size([143780, 3]),  self.mean_x=tensor([ 0.5044,  0.4678,  0.6521, -0.0124,  0.0204, -0.0232,  0.0282,  0.0481],
       device='cuda:0'), self.std_x=tensor([0.2904, 0.1506, 0.1444, 0.1219, 0.1065, 0.0854, 2.8319, 3.0344],
       device='cuda:0')
FluidSolidInterfaceData: self.txy_interface.shape=torch.Size([25452, 3]),  self.mean_x=tensor([ 5.0001e-01,  4.6873e-01,  6.5305e-01, -1.0230e-02,  1.8494e-02,
        -6.9603e-03, -7.7546e+01,  5.0739e+01], device='cuda:0'), self.std_x=tensor([2.9155e-01, 1.7971e-01, 1.7300e-01, 1.4451e-01, 1.3330e-01, 1.8181e-01,
        9.7830e+02, 7.2563e+02], device='cuda:0')

In [43]:
Fluid_data = scipy.io.loadmat(DATASET_PATH)
Fluid_training = Fluid_data["Fluid_training"]